In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import learning_curve
import numpy as np
df = pd.read_csv("../input/spaceship-titanic/train.csv")
df

 ******Explore Data**

In [ ]:
df['HomePlanet'].value_counts().plot.bar()
plt.xlabel('Planets')
plt.ylabel('values')

In [ ]:
df['CryoSleep'].value_counts().plot.bar()
plt.xlabel('CryoSleep')
plt.ylabel('values')

**#most passengers wanted to visit TRAPIST-1e**


In [ ]:
df['Destination'].value_counts().plot.bar(color=['red', 'green','blue']) 
plt.xlabel('Destination')
plt.ylabel('values')

**Most of the passengers are lightweight passengers**

In [ ]:
df['VIP'].value_counts().plot.bar()
plt.xlabel('VIP')
plt.ylabel('values')

# **MOST IMPORTENT #Data are balanced**

In [ ]:
df['Transported'].value_counts().plot.bar(color=['red', 'green']) 
plt.xlabel('Transported')
plt.ylabel('values')


In [ ]:
df["Transported"].value_counts().plot.pie()
plt.gca().set_aspect("equal")

***Most Transported passengers are from EUROPA*****

In [ ]:
sns.barplot(x = 'HomePlanet',y = 'Transported',hue = 'HomePlanet',data = df, palette = "Blues")
##Passengers from europa are the most Transported in the trip

# #checking missing values

In [ ]:
df.isna().any() 

# Missing Values Percentage%

In [ ]:
df.isna().sum().sum() /df.shape[0] *100 

# Filling Missing Data

In [ ]:
#Filling missing values
imp = SimpleImputer(strategy="mean")

df[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = pd.DataFrame(imp.fit_transform(df[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]))

imp_m = SimpleImputer(strategy="most_frequent")
df[['HomePlanet','CryoSleep','Cabin','Destination','VIP']] = pd.DataFrame(imp_m.fit_transform(df[['HomePlanet','CryoSleep','Cabin','Destination','VIP']]))

df.isna().any()

In [ ]:
X = df.drop(columns =['PassengerId','Name','Destination','Transported','Cabin'],axis = 1)
y = df['Transported']
X.head()

# #Data Scalling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_s = StandardScaler()


X = pd.get_dummies(X,columns=['HomePlanet','CryoSleep','VIP'])

X[['RoomService','Age','FoodCourt', 'ShoppingMall','Spa','VRDeck']] = pd.DataFrame(scaler_s.fit_transform(
    X[['RoomService','Age','FoodCourt', 'ShoppingMall','Spa','VRDeck']]))
X.head()

# #Splitting train data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8)

# RandomForest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model =RandomForestClassifier(n_estimators=150,criterion='gini',max_depth=90,max_leaf_nodes=120
                              ,verbose=1)

model.fit(X_train, y_train)
preds=model.predict(X_valid)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_valid, preds)
cnf_matrix

import matplotlib.pyplot as plt
import numpy as np
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


from sklearn.metrics import classification_report
print(classification_report(y_valid, preds))

# Public Score = 0.79354